***GENERATED CODE FOR frauddetectionclassypl PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = transformationDF.drop('Customer_ID')
        transformationDF = transformationDF.drop('Customer_Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Andaman and Nicobar Islands', 'max': 'West Bengal', 'missing': '0', 'distinct': '34'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'City', 'transformation_label': 'String Indexer'}], 'feature': 'City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Agartala', 'max': 'Ziro', 'missing': '0', 'distinct': '143'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'City'}, {'feature_label': 'City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Bank_Branch', 'transformation_label': 'String Indexer'}], 'feature': 'Bank_Branch', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Agartala Branch', 'max': 'Ziro Branch', 'missing': '0', 'distinct': '143'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Bank_Branch'}, {'feature_label': 'Bank_Branch', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Bank_Branch')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Account_Type', 'transformation_label': 'String Indexer'}], 'feature': 'Account_Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Business', 'max': 'Savings', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Account_Type'}, {'feature_label': 'Account_Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Account_Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Transaction_ID', 'transformation_label': 'String Indexer'}], 'feature': 'Transaction_ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '003d4818-8cb6-4bcf-aefc-03eb8c1d0619', 'max': 'fea3ca28-98a5-4ea8-9fb8-7f872cf23320', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Transaction_ID'}, {'feature_label': 'Transaction_ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Transaction_ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Transaction_Date', 'transformation_label': 'String Indexer'}], 'feature': 'Transaction_Date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '01-01-2025', 'max': '30-01-2025', 'missing': '0', 'distinct': '30'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Transaction_Date'}, {'feature_label': 'Transaction_Date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Transaction_Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Transaction_Time', 'transformation_label': 'String Indexer'}], 'feature': 'Transaction_Time', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '00:01:00', 'max': '23:55:18', 'missing': '0', 'distinct': '499'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Transaction_Time'}, {'feature_label': 'Transaction_Time', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Transaction_Time')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Transaction_Amount', 'threshold': 48577.578, 'transformation_label': 'Binarizer'}], 'feature': 'Transaction_Amount', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '48578.06', 'stddev': '27368.59', 'min': '47.39', 'max': '98375.96', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'Transaction_Amount'}, {'feature_label': 'Transaction_Amount', 'threshold': 48577.578, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('Transaction_Amount')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Merchant_ID', 'transformation_label': 'String Indexer'}], 'feature': 'Merchant_ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '000978b2-a429-4271-a275-3ca30ec71236', 'max': 'ffc9c2cc-5820-4b3a-beef-05c6ee3021e4', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Merchant_ID'}, {'feature_label': 'Merchant_ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Merchant_ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Transaction_Type', 'transformation_label': 'String Indexer'}], 'feature': 'Transaction_Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Bill Payment', 'max': 'Withdrawal', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Transaction_Type'}, {'feature_label': 'Transaction_Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Transaction_Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Merchant_Category', 'transformation_label': 'String Indexer'}], 'feature': 'Merchant_Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Clothing', 'max': 'Restaurant', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Merchant_Category'}, {'feature_label': 'Merchant_Category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Merchant_Category')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Account_Balance', 'threshold': 53246.96, 'transformation_label': 'Binarizer'}], 'feature': 'Account_Balance', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '51841.49', 'stddev': '27742.92', 'min': '5029.66', 'max': '99996.44', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'Account_Balance'}, {'feature_label': 'Account_Balance', 'threshold': 53246.96, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('Account_Balance')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Transaction_Device', 'transformation_label': 'String Indexer'}], 'feature': 'Transaction_Device', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ATM', 'max': 'Web Browser', 'missing': '0', 'distinct': '20'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Transaction_Device'}, {'feature_label': 'Transaction_Device', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Transaction_Device')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Transaction_Location', 'transformation_label': 'String Indexer'}], 'feature': 'Transaction_Location', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Agartala, Tripura', 'max': 'Ziro, Arunachal Pradesh', 'missing': '0', 'distinct': '146'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Transaction_Location'}, {'feature_label': 'Transaction_Location', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Transaction_Location')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Device_Type', 'transformation_label': 'String Indexer'}], 'feature': 'Device_Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ATM', 'max': 'POS', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Device_Type'}, {'feature_label': 'Device_Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Device_Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Transaction_Currency', 'transformation_label': 'String Indexer'}], 'feature': 'Transaction_Currency', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'INR', 'max': 'INR', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Transaction_Currency'}, {'feature_label': 'Transaction_Currency', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Transaction_Currency')
        transformationDF = transformationDF.drop('Customer_Contact')
        transformationDF = transformationDF.drop('Transaction_Description')
        transformationDF = transformationDF.drop('Customer_Email')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run frauddetectionclassyplHooks.ipynb
try:
	#sourcePreExecutionHook()

	bankfdtest = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/bankFD_test.csv', 'filename': 'bankFD_test.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(bankfdtest)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run frauddetectionclassyplHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(bankfdtest,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Customer_ID", "transformation_label": "String Indexer"}], "feature": "Customer_ID", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "00db9c1a-75c2-490a-9a85-f0250859d301", "max": "ff79f20e-c86a-45ba-a18e-e40eb6b9301c", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer_ID"}, {"transformationsData": [{"feature_label": "Customer_Name", "transformation_label": "String Indexer"}], "feature": "Customer_Name", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Aachal Din", "max": "Zinal Vaidya", "missing": "0", "distinct": "499"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer_Name"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "43.88", "stddev": "15.19", "min": "18", "max": "70", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Andaman and Nicobar Islands", "max": "West Bengal", "missing": "0", "distinct": "34"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"feature_label": "City", "transformation_label": "String Indexer"}], "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Agartala", "max": "Ziro", "missing": "0", "distinct": "143"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "City"}, {"transformationsData": [{"feature_label": "Bank_Branch", "transformation_label": "String Indexer"}], "feature": "Bank_Branch", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Agartala Branch", "max": "Ziro Branch", "missing": "0", "distinct": "143"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Bank_Branch"}, {"transformationsData": [{"feature_label": "Account_Type", "transformation_label": "String Indexer"}], "feature": "Account_Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Business", "max": "Savings", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Account_Type"}, {"transformationsData": [{"feature_label": "Transaction_ID", "transformation_label": "String Indexer"}], "feature": "Transaction_ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "003d4818-8cb6-4bcf-aefc-03eb8c1d0619", "max": "fea3ca28-98a5-4ea8-9fb8-7f872cf23320", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Transaction_ID"}, {"transformationsData": [{"feature_label": "Transaction_Date", "transformation_label": "String Indexer"}], "feature": "Transaction_Date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "01-01-2025", "max": "30-01-2025", "missing": "0", "distinct": "30"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Transaction_Date"}, {"transformationsData": [{"feature_label": "Transaction_Time", "transformation_label": "String Indexer"}], "feature": "Transaction_Time", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "00:01:00", "max": "23:55:18", "missing": "0", "distinct": "499"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Transaction_Time"}, {"transformationsData": [{"feature_label": "Transaction_Amount", "threshold": 48577.578, "transformation_label": "Binarizer"}], "feature": "Transaction_Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "48578.06", "stddev": "27368.59", "min": "47.39", "max": "98375.96", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "Transaction_Amount"}, {"transformationsData": [{"feature_label": "Merchant_ID", "transformation_label": "String Indexer"}], "feature": "Merchant_ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "000978b2-a429-4271-a275-3ca30ec71236", "max": "ffc9c2cc-5820-4b3a-beef-05c6ee3021e4", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Merchant_ID"}, {"transformationsData": [{"feature_label": "Transaction_Type", "transformation_label": "String Indexer"}], "feature": "Transaction_Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Bill Payment", "max": "Withdrawal", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Transaction_Type"}, {"transformationsData": [{"feature_label": "Merchant_Category", "transformation_label": "String Indexer"}], "feature": "Merchant_Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Clothing", "max": "Restaurant", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Merchant_Category"}, {"transformationsData": [{"feature_label": "Account_Balance", "threshold": 53246.96, "transformation_label": "Binarizer"}], "feature": "Account_Balance", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "51841.49", "stddev": "27742.92", "min": "5029.66", "max": "99996.44", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "Account_Balance"}, {"transformationsData": [{"feature_label": "Transaction_Device", "transformation_label": "String Indexer"}], "feature": "Transaction_Device", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ATM", "max": "Web Browser", "missing": "0", "distinct": "20"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Transaction_Device"}, {"transformationsData": [{"feature_label": "Transaction_Location", "transformation_label": "String Indexer"}], "feature": "Transaction_Location", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Agartala, Tripura", "max": "Ziro, Arunachal Pradesh", "missing": "0", "distinct": "146"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Transaction_Location"}, {"transformationsData": [{"feature_label": "Device_Type", "transformation_label": "String Indexer"}], "feature": "Device_Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ATM", "max": "POS", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Device_Type"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Is_Fraud", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.03", "stddev": "0.18", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Is_Fraud"}, {"transformationsData": [{"feature_label": "Transaction_Currency", "transformation_label": "String Indexer"}], "feature": "Transaction_Currency", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "INR", "max": "INR", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Transaction_Currency"}, {"transformationsData": [{"feature_label": "Customer_Contact", "transformation_label": "String Indexer"}], "feature": "Customer_Contact", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "+9191011XXXXXX", "max": "+9199998XXXXXX", "missing": "0", "distinct": "489"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer_Contact"}, {"transformationsData": [{"feature_label": "Transaction_Description", "transformation_label": "String Indexer"}], "feature": "Transaction_Description", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ATM withdrawal", "max": "Withdrawal", "missing": "0", "distinct": "162"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Transaction_Description"}, {"transformationsData": [{"feature_label": "Customer_Email", "transformation_label": "String Indexer"}], "feature": "Customer_Email", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "aachalXXXXX@XXXXXX.com", "max": "zehaanXXXXX@XXXXXX.com", "missing": "0", "distinct": "461"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer_Email"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run frauddetectionclassyplHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["Age", "Customer_ID_stringindexer", "Customer_Name_stringindexer", "Gender_stringindexer", "State_stringindexer", "City_stringindexer", "Bank_Branch_stringindexer", "Account_Type_stringindexer", "Transaction_ID_stringindexer", "Transaction_Date_stringindexer", "Transaction_Time_stringindexer", "Transaction_Amount_binarizer", "Merchant_ID_stringindexer", "Transaction_Type_stringindexer", "Merchant_Category_stringindexer", "Account_Balance_binarizer", "Transaction_Device_stringindexer", "Transaction_Location_stringindexer", "Device_Type_stringindexer", "Transaction_Currency_stringindexer", "Customer_Contact_stringindexer", "Transaction_Description_stringindexer", "Customer_Email_stringindexer"], "Is_Fraud")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

